In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacial_radiance as br
from datetime import datetime
import re

# testfolder = Path().resolve().parent.parent / 'TEMP' / 'bifacial_radiance'

testfolder = os.path.join('HPCruns', 'cchoi2', 'chongseok_2024-06-04_00h48', '2023-09-26__00_00_00', 'results')

# if not os.path.exists(testfolder):
#     os.makedirs(testfolder)
    
# print ("Your simulation will be stored in %s" % testfolder)

In [2]:
# remove unwanted results, in this case all the "fronts"
filesall = os.listdir(os.path.join(testfolder)) # How do I save the path to an object so that I don't have to repeat the whole process?

# filestoclean = [e for e in filesall_origin if e.endswith('_Front.csv')]
# for cc in range(0, len(filestoclean_origin)):
#     filetoclean_origin = filestoclean_origin[cc]
#     os.remove(os.path.join(testfolder,'BARN_validation_jan18_2023','results', filetoclean_origin))

In [3]:
print(filesall)


['irr_1axis_2023-09-26_0615Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S4_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S5_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S4_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S5_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S4_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S5_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0700Ground_S1_Scene0_Row5_Module

In [4]:
from datetime import datetime
import re

filestoclean = [e for e in filesall if e.endswith('_Back.csv')]
for cc in range(0, len(filestoclean)):
    filetoclean = filestoclean[cc]
    os.remove(os.path.join(testfolder, filetoclean))

filesall = os.listdir(os.path.join(testfolder))

## Read in the results
Below script reads in the results of the bifacial radiance modeling for a specific date and appends them to arrays that will then be joined into a dataframe.

In [5]:
# exampledata = br.load.read1Result(os.path.join(testfolder, 'irr_1axis_2023-09-26_0600Ground_S2_Scene0_Row5_Module10_Front.csv'))
exampledata = br.load.read1Result(os.path.join(testfolder, filesall[0]))
print(exampledata)



        x             y     z     rearZ      mattype                rearMat  \
0  0.3238  4.895009e-17  0.05  1.440969  groundplane  a9.4.a0.PVmodule.2310   
1  0.3238  2.540000e-02  0.05  1.455477  groundplane  a9.4.a0.PVmodule.2310   
2  0.3238  5.080000e-02  0.05  1.469986  groundplane  a9.4.a0.PVmodule.2310   
3  0.3238  7.620000e-02  0.05  1.484494  groundplane  a9.4.a0.PVmodule.2310   
4  0.3238  1.016000e-01  0.05  1.499003  groundplane  a9.4.a0.PVmodule.2310   
5  0.3238  1.270000e-01  0.05  1.513511  groundplane  a9.4.a0.PVmodule.2310   
6  0.3238  1.524000e-01  0.05  1.528019  groundplane  a9.4.a0.PVmodule.2310   
7  0.3238  1.778000e-01  0.05  1.542528  groundplane  a9.4.a0.PVmodule.2310   
8  0.3238  2.032000e-01  0.05  1.557036  groundplane  a9.4.a0.PVmodule.2310   

   Wm2Front   Wm2Back  Back/FrontRatio     rearX         rearY  
0  15.99923  4.727308         0.295453  0.797820  4.885237e-17  
1  15.99282  4.705595         0.294213  0.598347  3.663817e-17  
2  15.98670  4

In [6]:
prefixed = [filename for filename in filesall]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Modeled = []
Offset = []
Position = []
inchtom = 0.0254
print('{} files in the directory'.format(filesall.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.


ind = prefixed[0].split('_')
resultsDF = br.load.read1Result(os.path.join(testfolder, prefixed[0]))
datalength = len(resultsDF)
offsetlength = (datalength - 1)/2

print(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
print(resultsDF)
for ii in range (0, datalength):
    try:
        # Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
        print(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
        # Position.append(re.search(r'S(\d+)', prefixed[i])[1])
        Position.append(re.search(r'S(\d+)', prefixed[0])[1])
        # print(Position)
        # Offset.append(re.search(r'_offset_(-?\d+)', prefixed[i])[1])
        Offset.append(np.round(resultsDF['y'][ii]/0.0254).astype(int)-offsetlength)
        # print(Offset)
        # Modeled.append(resultsDF['Wm2Front'][ii])
        # print(Modeled)

        # filenamed.append(prefixed[0])
    except:
        print(" FAILED ", i, prefixed[0])
        faillist.append(prefixed[0])

225 files in the directory
225 groundscan files in the directory
2023-09-26_0615
        x             y     z     rearZ      mattype                rearMat  \
0  0.3238  4.895009e-17  0.05  1.440969  groundplane  a9.4.a0.PVmodule.2310   
1  0.3238  2.540000e-02  0.05  1.455477  groundplane  a9.4.a0.PVmodule.2310   
2  0.3238  5.080000e-02  0.05  1.469986  groundplane  a9.4.a0.PVmodule.2310   
3  0.3238  7.620000e-02  0.05  1.484494  groundplane  a9.4.a0.PVmodule.2310   
4  0.3238  1.016000e-01  0.05  1.499003  groundplane  a9.4.a0.PVmodule.2310   
5  0.3238  1.270000e-01  0.05  1.513511  groundplane  a9.4.a0.PVmodule.2310   
6  0.3238  1.524000e-01  0.05  1.528019  groundplane  a9.4.a0.PVmodule.2310   
7  0.3238  1.778000e-01  0.05  1.542528  groundplane  a9.4.a0.PVmodule.2310   
8  0.3238  2.032000e-01  0.05  1.557036  groundplane  a9.4.a0.PVmodule.2310   

   Wm2Front   Wm2Back  Back/FrontRatio     rearX         rearY  
0  15.99923  4.727308         0.295453  0.797820  4.885237e-17 

In [7]:
prefixed = [filename for filename in filesall]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Modeled = []
Offset = []
Position = []
inchtom = 0.0254
print('{} files in the directory'.format(filesall.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')
    resultsDF = br.load.read1Result(os.path.join(testfolder, prefixed[i]))
    datalength = len(resultsDF)
    offsetlength = (datalength - 1)/2
    for ii in range (0, datalength):
        try:
            Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
            # Position.append(re.search(r'S(\d+)', prefixed[i])[1])
            Position.append(re.search(r'S(\d+)', prefixed[i])[1])
            # Offset.append(re.search(r'_offset_(-?\d+)', prefixed[i])[1])
            Offset.append(np.round(resultsDF['y'][ii]/0.0254).astype(int)-offsetlength)
            Modeled.append(resultsDF['Wm2Front'][ii])

            filenamed.append(prefixed[i])
        except:
            print(" FAILED ", i, prefixed[i])
            faillist.append(prefixed[i])

results_compiled = pd.DataFrame(list(zip(Datetime, Position, Offset, Modeled)), columns = ['Datetime', "Position", "Offset", "Modeled"]) 
results_compiled['Datetime'] = pd.to_datetime(results_compiled['Datetime'], format ='%Y-%m-%d_%H%M')
print(results_compiled)

225 files in the directory
225 groundscan files in the directory
                Datetime Position  Offset   Modeled
0    2023-09-26 06:15:00        1    -4.0  15.99923
1    2023-09-26 06:15:00        1    -3.0  15.99282
2    2023-09-26 06:15:00        1    -2.0  15.98670
3    2023-09-26 06:15:00        1    -1.0  15.98059
4    2023-09-26 06:15:00        1     0.0  15.97447
...                  ...      ...     ...       ...
2020 2023-09-26 17:15:00        5     0.0  19.47111
2021 2023-09-26 17:15:00        5     1.0  19.46929
2022 2023-09-26 17:15:00        5     2.0  19.46746
2023 2023-09-26 17:15:00        5     3.0  19.46564
2024 2023-09-26 17:15:00        5     4.0  19.46382

[2025 rows x 4 columns]


In [8]:
# del(measured_l)
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1).rename(columns = {"Ap_1":"1", "Ap_2":"2", "Ap_3":"3", "Ap_4":"4", "Ap_5":"5"}) # The field pyranometer data
measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean().reset_index() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.
# print(measured)
measured_irrad = measured[["Datetime", "1", "2", "3", "4", "5"]]
measured_irrad = pd.melt(measured_irrad, id_vars = ["Datetime"], value_vars = ["1", "2", "3", "4", "5"], value_name = "Measured", var_name = "Position")
measured_irrad['Datetime'] = pd.to_datetime(measured_irrad['Datetime'], format ='%m/%d/%Y %H:%M')
# print(measured_irrad)
# combined = pd.concat([results_compiled, measured_irrad], axis = 0)#,keys= ["Datetime", "Position", "Offset"])#.set_index(["Datetime", "Position"])
# combined = pd.concat([results_compiled, measured_irrad.assign(Offset = -2), measured_irrad.assign(Offset = -1), measured_irrad.assign(Offset = 0), measured_irrad.assign(Offset = 1), measured_irrad.assign(Offset = 2)], join = "outer")#,keys= ["Datetime", "Position", "Offset"])
combined = pd.merge(results_compiled, measured_irrad).drop_duplicates(subset = ["Datetime", "Position", "Offset"])#, on = ["Datetime", "Position"])
combined['ModeledMinusMeasured'] = combined['Modeled'] - combined['Measured']
# results_compiled.merge()
print(combined)
# measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.
# measured_l = measured.reset_index()[["Datetime",'ap1', 'ap2', 'ap3', 'ap4', 'ap5']].melt(id_vars= "Datetime", value_vars = ['ap1', 'ap2', 'ap3', 'ap4', 'ap5'], value_name = "measured")
# measured_l_offset = pd.concat([measured_l.assign(offset = "west"), measured_l.assign(offset = "east"), measured_l.assign(offset = "origin")]).set_index(["Datetime", "variable", "offset"])

                Datetime Position  Offset   Modeled   Measured  \
0    2023-09-26 06:15:00        1    -4.0  15.99923  16.187596   
1    2023-09-26 06:15:00        1    -3.0  15.99282  16.187596   
2    2023-09-26 06:15:00        1    -2.0  15.98670  16.187596   
3    2023-09-26 06:15:00        1    -1.0  15.98059  16.187596   
4    2023-09-26 06:15:00        1     0.0  15.97447  16.187596   
...                  ...      ...     ...       ...        ...   
2020 2023-09-26 17:15:00        5     0.0  19.47111  17.048136   
2021 2023-09-26 17:15:00        5     1.0  19.46929  17.048136   
2022 2023-09-26 17:15:00        5     2.0  19.46746  17.048136   
2023 2023-09-26 17:15:00        5     3.0  19.46564  17.048136   
2024 2023-09-26 17:15:00        5     4.0  19.46382  17.048136   

      ModeledMinusMeasured  
0                -0.188366  
1                -0.194776  
2                -0.200896  
3                -0.207006  
4                -0.213126  
...                    ...  
2020

In [9]:
# Index the modeled data to the datetime column and add suffix before concatenating with the measured data. This will allow the "inner" join of the two dataframes using the indexed datetime and easy pivoting of the data afterwards.
# modeled = resultsdf.set_index('Datetime')
# modeled_suffix = modeled.add_suffix(".modeled")
# modeled['datatype']="modeled"

# Load the measured data
# measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
srrl_weather = pd.read_csv(str(Path().resolve() / 'WeatherFiles' /  'PSM3_15T.csv'), header = 2) # Data from SRRL

# Create a Datetime variable and set it as the index
srrl_weather['date'] = srrl_weather[['Year','Month','Day']].astype(str).agg('-'.join, axis=1) # Couldn't think of a proper way to chain this process into a single command. If you think of something, please let me know
srrl_weather['time'] = srrl_weather[['Hour','Minute']].astype(str).agg(':'.join, axis=1)
srrl_weather['Datetime'] = pd.to_datetime(srrl_weather[['date','time']].agg(" ".join, axis=1))
# srrl_weather.set_index('Datetime', inplace=True)

combined_ref = pd.merge(combined, srrl_weather.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute']))
print(combined_ref)



# Create a Datetime variable for the measured data using the existing 'TIMESTAMP' column.pivot

# measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
# measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
# measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.

# Add suffix to the columns in the measured dataframe before concatenating it with the modeled dataframe
# measured_suffix = measured.add_suffix(".measured")
# measured["datatype"]="measured"

# modeledandmeasured_suffix = pd.concat([modeled_suffix, measured_suffix], axis=1, join='inner')
# modeledandmeasured_melt = pd.wide_to_long(df = modeledandmeasured_suffix.reset_index(), stubnames= ["Ap_1", "Ap_2", "Ap_3", "Ap_4", "Ap_5","ST_con", "ST_pv_bp", "ST_pv_is", "PAR_con", "PAR_pv", "Batt_Volt", "PTemp_C"], i = "Datetime", j = "datatype", sep = ".", suffix = "\D+").reset_index().set_index('Datetime')

# modeledandmeasured_melt2 = modeledandmeasured_melt[['datatype', 'Ap_1', 'Ap_2', 'Ap_3', 'Ap_4', 'Ap_5']].reset_index().melt(id_vars = ['Datetime', 'datatype'], var_name = 'position', value_name= 'value').reset_index()
# modeledandmeasured_melt3 = modeledandmeasured_melt2.pivot(index=['Datetime', 'position'], columns = 'datatype', values = 'value')


# # Sum up the irradiance values and divide it by 4 to get accumulated daily radiation
# modeledandmeasured_sum = modeledandmeasured_melt2.reset_index().groupby(['position', 'datatype']).agg({"value":"sum"})['value'].transform(lambda x:x/4)

# # Grab just the measured
# ghipar = modeledandmeasured_melt.loc[modeledandmeasured_melt['datatype']=='measured'][['Ap_3', 'PAR_pv', 'PAR_con']]
# srrlandghipar = pd.concat([ghipar, srrl_weather], axis=1, join="inner")[['Ap_3','GHI', 'PAR_pv', 'PAR_con']].rename(columns={'Ap_3':'ghi_pv','GHI':'ghi_con','PAR_pv':'par_pv','PAR_con':'par_con'}).reset_index()
# srrlandghipar = pd.wide_to_long(df = srrlandghipar.reset_index(), stubnames=['ghi','par'], i = "Datetime", j = "location", sep = "_", suffix = "\D+").reset_index()
# print(srrlandghipar)


                Datetime Position  Offset   Modeled   Measured  \
0    2023-09-26 06:15:00        1    -4.0  15.99923  16.187596   
1    2023-09-26 06:15:00        1    -3.0  15.99282  16.187596   
2    2023-09-26 06:15:00        1    -2.0  15.98670  16.187596   
3    2023-09-26 06:15:00        1    -1.0  15.98059  16.187596   
4    2023-09-26 06:15:00        1     0.0  15.97447  16.187596   
...                  ...      ...     ...       ...        ...   
2020 2023-09-26 17:15:00        5     0.0  19.47111  17.048136   
2021 2023-09-26 17:15:00        5     1.0  19.46929  17.048136   
2022 2023-09-26 17:15:00        5     2.0  19.46746  17.048136   
2023 2023-09-26 17:15:00        5     3.0  19.46564  17.048136   
2024 2023-09-26 17:15:00        5     4.0  19.46382  17.048136   

      ModeledMinusMeasured      Wspd       Tdry        DHI         DNI  \
0                -0.188366  1.812800  13.884000  16.142607  180.110827   
1                -0.194776  1.812800  13.884000  16.142607 

In [10]:
# look at residuals (MBD, RMSE) based on Grear, Gpoa and Gtotal_modeled. From B. Marion Solar Energy 2016
def MBD(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*((1/m)*sum(df.model-df.meas))/df.meas.mean()
    return out

def RMSE(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*np.sqrt(1/m*sum((df.model-df.meas)**2))/df.meas.mean()
    return out

# residuals absolute output (not %) 
def MBD_abs(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = ((1/m)*sum(df.model-df.meas))
    return out

def RMSE_abs(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = np.sqrt(1/m*sum((df.model-df.meas)**2))
    return out

In [11]:
# Apply the RMSE and MBD functions by groups of position + offset
combined_mbd = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: MBD(x["Modeled"], x["Measured"])).reset_index(name = "mbd")
combined_mbdabs = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: MBD_abs(x["Modeled"], x["Measured"])).reset_index(name = "mbd_abs")
combined_rmse = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: RMSE(x["Modeled"], x["Measured"])).reset_index(name = "rmse")
combined_rmseabs = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: RMSE_abs(x["Modeled"], x["Measured"])).reset_index(name = "rmse_abs")

combined_validation = combined_mbd.merge(combined_mbdabs.merge(combined_rmse.merge(combined_rmseabs, on = ['Position', 'Offset']), on = ['Position', 'Offset']), on = ['Position', 'Offset'])
combined_validation["Offset"] = combined_validation["Offset"].astype("category")
# combined_validation["Offset"].cat.reorder_categories(["-2", "-1", "0", "1", "2"])
print(combined_validation)

   Position Offset        mbd    mbd_abs       rmse    rmse_abs
0         1   -4.0  11.764308  21.396725  67.452339  122.681180
1         1   -3.0  11.789197  21.437219  67.468587  122.683412
2         1   -2.0  11.814304  21.478050  67.485050  122.685792
3         1   -1.0  11.839426  21.518886  67.501529  122.688186
4         1    0.0  11.864554  21.559713  67.518027  122.690607
5         1    1.0  11.889696  21.600545  67.534550  122.693056
6         1    2.0  11.894544  21.608417  67.536815  122.691854
7         1    3.0  11.901941  21.620425  67.542756  122.694535
8         1    4.0   8.603591  16.103477  68.051651  127.373342
9         2   -4.0   5.342523  15.911368  36.100508  107.516325
10        2   -3.0   5.347860  15.926455  36.104351  107.522324
11        2   -2.0   5.353297  15.941825  36.108239  107.528353
12        2   -1.0   5.358734  15.957192  36.112128  107.534385
13        2    0.0   5.364173  15.972563  36.116021  107.540426
14        2    1.0   5.369610  15.987929

In [12]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

mbdplot = px.bar(combined_validation, x  = "Position", y = "mbd", color = "Offset", barmode = "group", width = 1000, height = 500)
mbdplot.show()

mbdabsplot = px.bar(combined_validation, x  = "Position", y = "mbd_abs", color = "Offset", barmode = "group" , width = 1000, height = 500)
mbdabsplot.show()

rmseplot = px.bar(combined_validation, x  = "Position", y = "rmse", color = "Offset", barmode = "group", width = 1000, height = 500)
rmseplot.show()

rmseabsplot = px.bar(combined_validation, x  = "Position", y = "rmse_abs", color = "Offset", barmode = "group", width = 1000, height = 500)
rmseabsplot.show()

# validplot = px.bar(combined_validation, x  = "Position", y = ["mbd", "mbd_abs", "rmse", "rmse_abs"], color = "Offset", facet_row = y,barmode = "group", width = 1000, height = 500)

fig1 = px.line(combined, x = 'Datetime', y = ["Modeled", "Measured"], facet_row = 'Position', facet_col = "Offset", width = 1000, height = 1000)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig1.show()


fig2 = px.scatter(combined, x="Modeled", y="Measured", trendline="ols", facet_col = "Offset", facet_row = "Position", hover_data = combined.columns[:], width = 1000, height = 1000)
fig2.show()

fig3 = px.line(combined, x = 'Datetime', y = ['Modeled', 'Measured', 'ModeledMinusMeasured'], facet_row = 'Position', facet_col = "Offset", width = 1000, height = 1000)
fig3.update_traces(opacity = 0.5)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.show()

In [13]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")


figures_to_html([mbdplot, mbdabsplot, rmseplot, rmseabsplot, fig1, fig2, fig3], 'BARN_validation_4inchoffsets.html')

## The following figures are best displayed in wide-screen.

In [14]:
# Set the size for each figure, in pixels
figwidth = 1000
figheight = 400



# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line(modeledandmeasured_melt2, x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = 1000)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig1.show()

rsqval = []
# 1-to-1 plot of the measured vs modeled irradiance values
fig2 = px.scatter(modeledandmeasured_melt3.reset_index(), x = 'measured', y = 'modeled', facet_col = 'position', facet_col_wrap=5, trendline="ols", trendline_color_override="black", width = figwidth, height = figheight)
    # Accessing the R-squared values and annotating each facet
for i in range(len(np.unique(modeledandmeasured_melt3.reset_index()['position']))): #'position' can be replaced with whatever column is being used for the facets
    # Accessing R-squared value
    rsqval=round(px.get_trendline_results(fig2).iloc[i]["px_fit_results"].rsquared, 4)

    # Annotating each facet with the R-squared value
    fig2.add_annotation(
        xref='paper', yref='paper',
        x=10, y=800,
        xanchor='left', yanchor='bottom',
        text=f'R<sup>2</sup>: {rsqval:.4f}',
        showarrow=False,
        font=dict(size=12),
        row= 1, col=i+1 # only works when the facets are in a single row
    )

fig2.update_layout(
    plot_bgcolor='white',
    xaxis_title = "Measured irradiance (W/m<sup>2</sup>)",
    yaxis_title = "Modeled irradiance (W/m<sup>2</sup>)"
)
fig2.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig2.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig2.update_traces(marker=dict(size=6,
                                color='rgba(135, 206, 250, 0.5)',
                                line=dict(width=2,
                                    color='DarkSlateGrey')),
                  selector=dict(mode='markers'))
fig2.show()

# side-by-side barcharts
fig3 = px.bar(modeledandmeasured_sum.reset_index(), x='position', y='value', color='datatype', barmode='group', width = figwidth, height = figheight)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "Daily total radiation (Wh/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.show()

# GHI vs PAR inside an array vs 
fig4 = px.scatter(srrlandghipar, x = 'ghi', y = 'par', color = 'location', trendline="ols",color_discrete_sequence=px.colors.qualitative.Antique, width = figwidth, height = figheight)

fig4.update_layout(
    legend=dict(
        yanchor="top",
        y=0.95,
        xanchor="left",
        x=0.01
        ),
    legend_title=dict(
        text = "Location"
        ),
    legend_orientation="h",
    plot_bgcolor='white',
    yaxis_title = "Measured photoactive radiation (umol m<sup>-2</sup> s<sup>-1</sup>)",
    xaxis_title = "Measured global horizontal irradiance (W/m<sup>2</sup>)"
)
fig4.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig4.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig4.update_traces(marker=dict(size=6,
                                # color='rgba(135, 206, 250, 0.5)',
                                line=dict(width=2,
                                    color='DarkSlateGrey')),
                  selector=dict(mode='markers'))


NameError: name 'modeledandmeasured_melt2' is not defined

## Stitch all the figures into a single HTML
Function below taken from https://stackoverflow.com/questions/45577255/plot-multiple-figures-as-subplots

In [ ]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")


figures_to_html([fig1, fig2, fig3, fig4], 'BARN_validation.html')

In [ ]:
print(modeledandmeasured_melt2)

In [ ]:
# Simple line graph of the measured vs modeled irradiance values
fig1 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_1'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig1.show()

fig2 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_2'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig2.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig2.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig2.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig2.show()

fig3 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_3'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig3.show()

fig4 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_4'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig4.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig4.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig4.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig4.show()

fig5 = px.line(modeledandmeasured_melt2.loc[modeledandmeasured_melt2['position'] == 'Ap_5'], x = 'Datetime', y = 'value', color = 'datatype', facet_row = 'position', facet_col_wrap = 1, width = figwidth, height = figheight)
fig5.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig5.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig5.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
# for axis in fig1.layout:
#    if axis.startswith("xaxis"):
#       fig1.layout[axis].title = ""
fig5.show()

In [ ]:
type(fig1.layout)

Next steps:<br>
Divide the data by AM and PM and <br>
Implement 15-minute averaging to the data: dataframe.resample(freq='15T', label='right', closed='right').mean() <br>
Keep the data in wide form to avoid repeating indices <br>
Look for the script MBD_RMSE_functions.py and perform error analysis to annotatet the scatterplot (also ) <br>
Plot Irradiance sensor 3 vs PAR sensor (for within row and in control (GHI from SRRL)) and use the time of the day as a gradient and point shape as the data type <br>
Superimpose some of the figures with a side-view of an agrivoltaics row